In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('combined_fall_data.csv')

In [3]:
df

,TimeStamp(s),FrameCounter,AccX,AccY,AccZ,GyrX,GyrY,GyrZ,EulerX,EulerY,EulerZ,Subject,Task,Trial,Fall,Fall Description
0,1.30,130.0,-0.043,-0.729,-0.159,25.439335,0.401071,-0.802141,19.033665,-12.163898,51.995939,6.0,20.0,1.0,1,Forward fall when trying to sit down
1,1.31,131.0,-0.043,-0.719,-0.142,27.731167,0.171887,-1.718874,19.314414,-12.152439,51.978750,6.0,20.0,1.0,1,Forward fall when trying to sit down
2,1.32,132.0,-0.042,-0.709,-0.136,29.965703,2.750198,-2.005353,19.623811,-12.118062,51.967291,6.0,20.0,1.0,1,Forward fall when trying to sit down
3,1.33,133.0,-0.049,-0.711,-0.157,31.971056,2.750198,-0.974029,19.944668,-12.083684,51.967291,6.0,20.0,1.0,1,Forward fall when trying to sit down
4,1.34,134.0,-0.062,-0.726,-0.151,33.575339,2.005353,0.401071,20.282713,-12.060766,51.978750,6.0,20.0,1.0,1,Forward fall when trying to sit down
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443026,8.16,816.0,-0.035,-1.004,0.138,-0.229183,-1.661578,2.578311,98.033114,-3.420559,-29.742250,38.0,36.0,5.0,0,Walk upstairs and downstairs quickly (5 steps)
2443027,8.17,817.0,-0.030,-1.000,0.135,-0.630254,-1.375099,2.463719,98.027384,-3.443478,-29.759439,38.0,36.0,5.0,0,Walk upstairs and downstairs quickly (5 steps)
2443028,8.18,818.0,-0.024,-0.997,0.131,-1.203212,-1.546987,2.349128,97.947170,-3.374723,-29.776627,38.0,36.0,5.0,0,Walk upstairs and downstairs quickly (5 steps)
2443029,8.19,819.0,-0.019,-0.997,0.126,-1.776170,-1.776170,2.349128,97.929981,-3.391911,-29.799546,38.0,36.0,5.0,0,Walk upstairs and downstairs quickly (5 steps)


In [4]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
# Separate features and target
X = df[['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ']]
#y = df['Fall Description']  # Replace 'activity' with the actual name of the target column
y = df['Fall'].astype(int)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Check the class distribution after applying SMOTE
print("Class distribution before SMOTE:")
print(y_train.value_counts())
print("\nClass distribution after SMOTE:")
print(y_train_smote.value_counts())

# Optionally, you can save the new balanced dataset
balanced_data = pd.concat([X_train_smote, y_train_smote], axis=1)
balanced_data.to_csv('bal_fall_detection.csv', index=False)

Class distribution before SMOTE:
Fall
0    1815754
1     138670
Name: count, dtype: int64

Class distribution after SMOTE:
Fall
0    1815754
1    1815754
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Handle missing values (if any)
df=balanced_data
df = df.dropna()

# Select features and target
features = ['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ']
X = df[features]
y = df['Fall'].astype(int)

# Normalize the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Feature importances
feature_importances = model.feature_importances_
for feature, importance in zip(features, feature_importances):
    print(f"{feature}: {importance}")

# Display feature importances in a bar plot
import matplotlib.pyplot as plt

plt.bar(features, feature_importances)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.show()